## Import packages and environment set up

In [ ]:
# import libraries
import os
import numpy as np
import pandas as pd
from tqdm import tqdm
import json
import seaborn as sns
import spacy
import wordcloud
import matplotlib.pyplot as plt
import plotly.express as px
import warnings
warnings.filterwarnings("ignore")
import torch
from torch.utils.data import Dataset
import sys
import pathlib
from collections import Counter
from typing import Callable
import jieba  
import codecs 
from wordcloud import WordCloud, STOPWORDS
import torch
import torch.nn as nn
import torch.nn.functional as F
pd.set_option("max_columns", None) # show all cols
pd.set_option('max_colwidth', None) 

In [ ]:
# mount to our google drive path
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# check if GPU is being used
# return True if GPU is being used
torch.cuda.is_available()

True

## Config

In [ ]:
os.listdir('/content/drive/MyDrive/GU/580/Data/')

['clothing_1w_1.json',
 'clothing_1w_2.json',
 'clothing_1w_3.json',
 'HIT_stop_words.txt',
 'photo',
 'ENG_clothing_1w_1.json',
 'ENG_clothing_1w_2.json',
 'ENG_clothing_1w_3.json',
 'img_embedding',
 'stop_words_english.txt',
 'model_save',
 'runs',
 'New_ENG_clothing_1w_1.json']

In [ ]:
root_path = os.path.abspath(os.path.dirname('/content/drive/MyDrive/GU/580/Data/'))
os.chdir(root_path)
text_files = ['clothing_1w_1.json','clothing_1w_2.json','clothing_1w_3.json']
# image embedding config
img_vecs_path = os.path.join(root_path, 'img_embedding/img_embedding_ResNet101.txt')
# save path
save_path = os.path.join(root_path, 'data_pairs.txt')
train_save_path = os.path.join(root_path, 'train.txt')
dev_save_path = os.path.join(root_path, 'dev.txt')
test_save_path = os.path.join(root_path, 'test.txt')

## Data preprocessing

<p> data preprocessing is to make src-tgt pairs and create train (80%), test(10%), validation(10%) split.

In [ ]:
def train_test_split(data_pairs, test_ratio = 0.1, dev_ratio = 0.1, train_save_path = train_save_path, dev_save_path = dev_save_path, test_save_path = test_save_path):
  """ split a whole sample set into training set (80%), dev set (20%) and test set (10%)
  """
  train, dev, test = [], [], []
  count = 0
  for data in data_pairs:
    if count <= int(len(data_pairs) * test_ratio):
      test.append(data)
    elif count <= int(len(data_pairs)* (test_ratio + dev_ratio)):
      dev.append(data)
    else:
      train.append(data)
    count += 1

  write_data(train, train_save_path)
  write_data(dev, dev_save_path)
  write_data(test, test_save_path)

  print('train: ', len(train))
  print('test: ', len(test))
  print('dev: ', len(dev))

In [ ]:
def write_data(text_data, file_path):
  """write the text data into a file.
  """
  with open(file_path, 'w', encoding = 'utf8') as file:
    for line in text_data:
      file.write(line)
      file.write('\n')

In [ ]:
def json_to_df(json_file):
  """turn json file to dataframe
  """
  with open(json_file, 'r', encoding = 'utf8') as file:
    js_data = json.load(file)
  df = pd.DataFrame.from_dict({i: js_data[i] 
                                  for i in js_data.keys()},
                                  orient='index').reset_index()
  return df

In [ ]:
def merge_json_df(files):
  dfs = []
  for f in files:
      json_path = os.path.join(root_path, f)
      df = json_to_df(json_path)
      dfs.append(df)
  merged_df = pd.concat(dfs)
  return merged_df

In [ ]:
# remove special characters in labels
import re
def clear_characters(text):
  text = re.sub('\[', '', text)
  text = re.sub('\]', '', text)
  text = re.sub('\'', '', text)
  return text

In [ ]:
import re
def get_img_vecs (img_vecs_path):
  img_vecs = {}
  with open(img_vecs_path, 'r') as file:
      for line in file:
          img_index, img_vec = line.strip().split('\t')
          img_index = re.sub(r"\.jpg","",img_index)
          # img_vec = re.sub(r"\[","",img_vec)
          # img_vec = re.sub(r"\]","",img_vec)
          # img_vec = list(map(lambda x: float(x), img_vec.split(', ')))
          img_vecs[img_index] = img_vec
  return img_vecs

In [ ]:
def text_img_concat_save(text_files, img_vecs_path, save_path):
  img_vecs = get_img_vecs(img_vecs_path)

  merged_df = merge_json_df(text_files)
  merged_df = merged_df.reset_index(drop = True)
  merged_df['tgt'] = merged_df['tgt'].astype('str')
  merged_df['tgt'] = merged_df['tgt'].apply(clear_characters)
  merged_df['img_vecs'] = ', '.join(['0']*1000)

  for key in tqdm(img_vecs.keys()):
    vecs = img_vecs.get(key)
    vecs = re.sub(r"\[","",vecs)
    vecs = re.sub(r"\]","",vecs)
    merged_df.loc[merged_df['index'] == key, 'img_vecs'] = vecs

  merged_df['new'] = merged_df['index'] + '<sep>' + merged_df['src'] + '<sep>' + merged_df['tgt'] + '<sep>' + merged_df['img_vecs']

  write_data(merged_df['new'], save_path)

  return merged_df['new']

In [ ]:
save_path

'/content/drive/MyDrive/GU/580/Data/data_pairs.txt'

In [ ]:
dataset = text_img_concat_save(text_files, img_vecs_path, save_path)

100%|██████████| 29832/29832 [01:01<00:00, 482.11it/s]


In [ ]:
train_test_split(dataset, test_ratio = 0.1, dev_ratio =0.1)

train:  7999
test:  1001
dev:  1000
